In [9]:
import warnings
warnings.filterwarnings('ignore')

In [10]:
# mysql에 저장한 데이터를 불러오기

import mysql.connector
import pandas as pd
import numpy as np

import mysql.connector
import pandas as pd

# MySQL 연결 설정
cnx = mysql.connector.connect(
    host='',
    user='',
    password='',
    database='trainDB'
)

# 쿼리 생성
query_ball = "SELECT * FROM ball_markup_1"
query_event = "SELECT * FROM events_markup_1"

# DataFrame으로 읽기
ball_df = pd.read_sql(query_ball, con=cnx)
events_df = pd.read_sql(query_event, con=cnx)

# 연결 종료 (선택적)
cnx.close()

In [13]:
events_df = events_df.drop(columns='level_0')
events_df

,frame,event
0,18,bounce
1,33,net
2,44,bounce
3,58,empty_event
4,98,net
...,...,...
599,79276,net
600,79291,bounce
601,79323,empty_event
602,79361,empty_event


In [11]:
# ball_df = ball_df.drop(columns='level_0')
# ball_df

,frame,x,y
0,14,506,523
1,15,523,529
2,16,537,534
3,17,552,541
4,18,566,544
...,...,...,...
6794,79412,852,617
6795,79413,-1,-1
6796,79414,-1,-1
6797,79415,853,616


[탁구 데이터세트](https://lab.osai.ai/)에서 불러온 데이터 세트는 y축이 반전되있다는 문제가 있습니다. 따라서 y축을 재반전 시키고 이를 **ball_df** 데이터세트에 적용합니다.

In [5]:
# y의 최대값
max_y = ball_df['y'].max()

# y축 값 뒤집기
ball_df['y'] = max_y - ball_df['y']

ball_df.head()

,level_0,frame,x,y
0,0,14,506,221
1,1,15,523,215
2,2,16,537,210
3,3,17,552,203
4,4,18,566,200


**ball_df**와 **events_df**를 합칩니다.

In [7]:
# ball_df와 events_df를 frame의 교집합 반환한 merged_df 생성

# events_df에 존재하는 프레임 값을 기준으로 ball_df를 필터링
filtered_df = ball_df[ball_df['frame'].isin(events_df['frame'])]

# 'frame' 열을 기준으로 events_df에서 filter_df로 'event' 열 병합
merged_df = pd.merge(filtered_df, events_df[['frame', 'event']], on='frame', how='left')

# 열을 재정렬하여 'y'와 'player' 사이에 'event'를 배치
merged_df = merged_df[['frame', 'x', 'y', 'event']]

# bounce,net 외 empty_event라는 이벤트도 있는데 이는 분석에 필요가 없으니 제외
# event에는 (bounce, net, empty_event) 3가지 유형이 있는데 empty_event의 필요성 불명
merged_df = merged_df[merged_df['event'] != 'empty_event']
merged_df.head()

,frame,x,y,event
0,18,566,200,bounce
1,33,897,228,net
2,44,500,138,bounce
3,98,886,442,net
4,138,1322,192,bounce


시각화하여 분포가 어떻게 이뤄져있는지 파악하고 y축 반전이 잘 되었나 확인합니다.\
\
테이블과 네트의 좌표는 동영상에서 미리 추출한 좌표를 사용합니다.

In [9]:
# 테이블 꼭지점 좌표
table_corners = {
    'left_bottom': (305, 641),
    'left_top': (510, 527),
    'right_top': (1297, 542),
    'right_bottom': (1435, 668)
}

# 테이블 꼭지점 좌표도 y좌표를 반전
table_corners = {corner: (x, max_y - y) for corner, (x, y) in table_corners.items()}

table_corners

{'left_bottom': (305, 103),
 'left_top': (510, 217),
 'right_top': (1297, 202),
 'right_bottom': (1435, 76)}

In [10]:
# 네트 좌표
net_coords = [(901, 534), (862, 655)]

# Flip the y-coordinates of the additional points
net_coords = [(x, max_y - y) for x, y in net_coords]

net_coords

[(901, 210), (862, 89)]

y축을 변환해준 데이터를 mysql로 보내고 EDA를 이어서 진행합니다.

In [12]:
# mysql로 보내기

from sqlalchemy import create_engine

db_connection_path = ''
db_connection = create_engine(db_connection_path)

conn = db_connection.connect()

ball_df.to_sql('ball_markup_1', con = db_connection, if_exists= 'replace')
events_df.to_sql('events_markup_1', con = db_connection, if_exists= 'replace')



604